<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/main/notebooks/Analysis%20LA%20Get%202019%20Social%20Distancing%20Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis LA Get 2019 Social Distancing Statistics

**Instructions:**

1. Mount the google drive to access the data by following **Steps to Mount the Drive**.
2. Provide the drive file path to the data, city name, and year in the fourth code cell. 

**Steps to Mount the Drive:**

1. Execute the first code cell.
2. There will be a link to follow in order to authorize the google account for drive. Go to that link.
3. A code to authorize the google account will be generated. Copy the code generated.
4. Go back to the cell where the process of mounting the drive is running. Paste the generated code from step 3 to the text box in the cell and press enter.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pathlib import Path

city = 'Los Angeles'
city_directory = 'la'
year = '2019'

# Provide a list of CSV file paths to read
path = Path('drive/MyDrive/big-data-project/data/clean-data/')
path_list = [path/city_directory/'social'/year]

In [ ]:
pip install geopandas

     |████████████████████████████████| 1.0MB 9.3MB/s 
     |████████████████████████████████| 6.5MB 13.1MB/s 
     |████████████████████████████████| 15.3MB 52.2MB/s 


In [ ]:
import os
import geopandas
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

import altair as alt
from vega_datasets import data

# # Allow altair to visualize large datasets
# alt.data_transformers.disable_max_rows()

Mounted at /content/drive


In [ ]:
#Appending into dataframe

def get_df(path_list):
    df = []
    first = True
    for directory in tqdm(path_list, desc='Years'):
        f_list = os.listdir(directory)
        csv_files = [directory/f for f in f_list if f.endswith('.csv')]
        for csv in tqdm(csv_files, desc='Days'):
            if first:
                df = pd.read_csv(csv, dtype={'cbg': object})
                first = False
            else:
                df = df.append(pd.read_csv(csv, dtype={'cbg': object}), ignore_index=True)
    return df.sort_values(by=['date_range_start'])

In [ ]:
%%time

df = get_df(path_list)



CPU times: user 12min 14s, sys: 22.9 s, total: 12min 37s
Wall time: 19min 53s


In [ ]:
df

,Unnamed: 0,cbg,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home,mean_home_dwell_time,mean_non_home_dwell_time,mean_distance_traveled_from_home
0,17,060371012201,2019-01-01T00:00:00-08:00,2019-01-02T00:00:00-08:00,213,4948,"{""16001-50000"":23,""0"":103,"">50000"":9,""2001-800...","{""16001-50000"":69,"">50000"":180,""2001-8000"":27,...",100,206,"{""721-1080"":24,""361-720"":20,""61-360"":34,""<60"":...","[68,68,77,77,74,75,78,78,78,79,79,71,70,70,59,...",10,1,"{""060371046201"":2,""060371012202"":1,""0605909980...",3,5,872,"{""21-45"":14,""481-540"":4,""541-600"":2,""46-60"":1,...",98,"{""0-25"":64,""76-100"":126,""51-75"":19,""26-50"":3}",469,129,43624
5495,147371,060372656011,2019-01-01T00:00:00-08:00,2019-01-02T00:00:00-08:00,60,3016,"{""16001-50000"":4,""0"":20,"">50000"":8,""2001-8000""...","{""16001-50000"":29,"">50000"":306,""<1000"":294,""20...",21,916,"{""721-1080"":14,""361-720"":9,""61-360"":8,""<60"":6,...","[23,31,33,35,32,34,34,34,33,30,33,33,30,30,30,...",5,2,"{""060377019022"":1,""060372656011"":50,""060372691...",2,83,115,"{""21-45"":1,""541-600"":1,""46-60"":3,""721-840"":2,""...",89,"{""0-25"":9,""76-100"":37,""51-75"":5,""26-50"":1}",764,277,23331
5494,147370,060372282101,2019-01-01T00:00:00-08:00,2019-01-02T00:00:00-08:00,104,1038,"{""16001-50000"":8,""0"":50,"">50000"":9,""2001-8000""...","{""16001-50000"":58,"">50000"":82,""<1000"":82,""2001...",48,675,"{""721-1080"":17,""361-720"":16,""61-360"":14,""<60"":...","[43,53,56,57,58,56,60,61,51,56,53,49,45,44,43,...",7,1,"{""060372283102"":3,""060590744061"":1,""0603754220...",1,17,316,"{""21-45"":3,""481-540"":1,""541-600"":3,""46-60"":2,""...",97,"{""0-25"":21,""76-100"":65,""51-75"":11,""26-50"":3}",683,186,41700
5493,147369,060371211011,2019-01-01T00:00:00-08:00,2019-01-02T00:00:00-08:00,59,1605,"{""16001-50000"":4,""0"":26,"">50000"":3,""2001-8000""...","{""16001-50000"":74,"">50000"":275,""<1000"":101,""20...",25,811,"{""721-1080"":11,""361-720"":8,""61-360"":5,""<60"":11...","[22,27,30,34,36,37,34,37,32,30,32,34,31,30,31,...",1,1,"{""060371013002"":1,""060379301011"":1,""0603712212...",1,24,172,"{""21-45"":5,""481-540"":1,""46-60"":1,""721-840"":4,""...",97,"{""0-25"":10,""76-100"":40,""51-75"":6,""26-50"":2}",737,156,7502
5492,147180,060590993102,2019-01-01T00:00:00-08:00,2019-01-02T00:00:00-08:00,70,2218,"{""16001-50000"":5,""0"":23,"">50000"":6,""2001-8000""...","{""16001-50000"":21,"">50000"":165,""<1000"":35,""200...",24,1102,"{""721-1080"":10,""361-720"":8,""61-360"":1,""<60"":11...","[41,44,46,49,47,45,47,46,45,45,43,45,45,45,40,...",2,1,"{""060590992451"":1,""060590993061"":1,""0605909970...",1,49,180,"{""21-45"":3,""481-540"":1,""46-60"":3,""1201-1320"":1...",96,"{""0-25"":9,""76-100"":47,""51-75"":10}",923,171,5418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999275,72383,060372404021,2019-12-31T00:00:00-08:00,2020-01-01T00:00:00-08:00,58,2700,"{""16001-50000"":8,""0"":9,"">50000"":3,""2001-8000"":...","{""16001-50000"":79,"">50000"":75,""<1000"":14,""2001...",8,567,"{""721-1080"":8,""361-720"":13,""61-360"":5,""<60"":7,...","[33,33,36,36,38,34,33,29,24,25,21,22,22,20,20,...",1,1,"{""060376500014"":1,""060372371011"":1,""0605900130...",3,122,164,"{""21-45"":3,""481-540"":1,""541-600"":2,""46-60"":4,""...",81,"{""0-25"":8,""76-100"":31,""51-75"":10,""26-50"":6}",677,199,3620
2999274,72382,060372404013,2019-12-31T00:00:00-08:00,2020-01-01T00:00:00-08:00,63,2489,"{""16001-50000"":6,""0"":27,"">50000"":1,""2001-8000""...","{""16001-50000"":67,"">50000"":240,""<10

In [ ]:
# Selecting the columsn that we need for the analysis
# col = ['date_range_start',
#        'cbg',
#        'device_count',  # Total active devices
#        'completely_home_device_count',
#        'part_time_work_behavior_devices',
#        'full_time_work_behavior_devices',
#        'delivery_behavior_devices',
#        'median_percentage_time_home',
#        'median_home_dwell_time',
#        'mean_home_dwell_time',
#        'median_non_home_dwell_time',
#        'mean_non_home_dwell_time',
#        'distance_traveled_from_home', # Median Distance traveled
#        'mean_distance_traveled_from_home']
col = ['date_range_start',
       'cbg',
       'device_count',  # Total active devices
       'completely_home_device_count',
       'part_time_work_behavior_devices',
       'full_time_work_behavior_devices']
devices_mobility = df[col]

In [ ]:
#Renaming columns
devices_mobility = devices_mobility.rename(columns={'date_range_start':'date','device_count':'total', 'completely_home_device_count':'completely_home','part_time_work_behavior_devices':'part_time_work', 'full_time_work_behavior_devices':'full_time_work'})
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work
0,2019-01-01T00:00:00-08:00,060371012201,213,100,10,1
5495,2019-01-01T00:00:00-08:00,060372656011,60,21,5,2
5494,2019-01-01T00:00:00-08:00,060372282101,104,48,7,1
5493,2019-01-01T00:00:00-08:00,060371211011,59,25,1,1
5492,2019-01-01T00:00:00-08:00,060590993102,70,24,2,1
...,...,...,...,...,...,...
2999275,2019-12-31T00:00:00-08:00,060372404021,58,8,1,1
2999274,2019-12-31T00:00:00-08:00,060372404013,63,27,2,1
2999273,2019-12-31T00:00:00-08:00,060371380001,82,22,10,2
2999239,2019-12-31T00:00:00-08:00,060374026004,75,25,4,1


In [ ]:
# devices_mobility

In [ ]:
#Listing out the device count columns
device_columns = ['completely_home','part_time_work','full_time_work']

In [ ]:
#Calculating percentage for all the device columns
for column in device_columns :
    devices_mobility['percentage_'+column]= devices_mobility.apply(lambda row: row[column]/row['total']*100.0 if row['total']!=0.0 else 0.0, axis=1)
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work
0,2019-01-01T00:00:00-08:00,060371012201,213,100,10,1,46.948357,4.694836,0.469484
5495,2019-01-01T00:00:00-08:00,060372656011,60,21,5,2,35.000000,8.333333,3.333333
5494,2019-01-01T00:00:00-08:00,060372282101,104,48,7,1,46.153846,6.730769,0.961538
5493,2019-01-01T00:00:00-08:00,060371211011,59,25,1,1,42.372881,1.694915,1.694915
5492,2019-01-01T00:00:00-08:00,060590993102,70,24,2,1,34.285714,2.857143,1.428571
...,...,...,...,...,...,...,...,...,...
2999275,2019-12-31T00:00:00-08:00,060372404021,58,8,1,1,13.793103,1.724138,1.724138
2999274,2019-12-31T00:00:00-08:00,060372404013,63,27,2,1,42.857143,3.174603,1.587302
2999273,2019-12-31T00:00:00-08:00,060371380001,82,22,10,2,26.829268,12.195122,2.439024
2999239,2019-12-31T00:00:00-08:00,060374026004,75,25,4,1,33.333333,5.333333,1.333333


In [ ]:
#Normalising the percentage columns
for column in device_columns:
        devices_mobility['norm_'+column] = (devices_mobility['percentage_'+ column] - devices_mobility['percentage_'+column].mean()) / devices_mobility['percentage_'+column].std()

devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
0,2019-01-01T00:00:00-08:00,060371012201,213,100,10,1,46.948357,4.694836,0.469484,1.654628,-0.885396,-1.218549
5495,2019-01-01T00:00:00-08:00,060372656011,60,21,5,2,35.000000,8.333333,3.333333,0.478038,-0.229283,-0.492229
5494,2019-01-01T00:00:00-08:00,060372282101,104,48,7,1,46.153846,6.730769,0.961538,1.576391,-0.518266,-1.093756
5493,2019-01-01T00:00:00-08:00,060371211011,59,25,1,1,42.372881,1.694915,1.694915,1.204068,-1.426358,-0.907759
5492,2019-01-01T00:00:00-08:00,060590993102,70,24,2,1,34.285714,2.857143,1.428571,0.407700,-1.216779,-0.975308
...,...,...,...,...,...,...,...,...,...,...,...,...
2999275,2019-12-31T00:00:00-08:00,060372404021,58,8,1,1,13.793103,1.724138,1.724138,-1.610269,-1.421088,-0.900348
2999274,2019-12-31T00:00:00-08:00,060372404013,63,27,2,1,42.857143,3.174603,1.587302,1.251754,-1.159533,-0.935052
2999273,2019-12-31T00:00:00-08:00,060371380001,82,22,10,2,26.829268,12.195122,2.439024,-0.326558,0.467095,-0.719040
2999239,2019-12-31T00:00:00-08:00,060374026004,75,25,4,1,33.333333,5.333333,1.333333,0.313916,-0.770259,-0.999462


In [ ]:
#Converting datetime format
devices_mobility['date'] = pd.to_datetime(devices_mobility['date'],utc= True)
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
0,2019-01-01 08:00:00+00:00,060371012201,213,100,10,1,46.948357,4.694836,0.469484,1.654628,-0.885396,-1.218549
5495,2019-01-01 08:00:00+00:00,060372656011,60,21,5,2,35.000000,8.333333,3.333333,0.478038,-0.229283,-0.492229
5494,2019-01-01 08:00:00+00:00,060372282101,104,48,7,1,46.153846,6.730769,0.961538,1.576391,-0.518266,-1.093756
5493,2019-01-01 08:00:00+00:00,060371211011,59,25,1,1,42.372881,1.694915,1.694915,1.204068,-1.426358,-0.907759
5492,2019-01-01 08:00:00+00:00,060590993102,70,24,2,1,34.285714,2.857143,1.428571,0.407700,-1.216779,-0.975308
...,...,...,...,...,...,...,...,...,...,...,...,...
2999275,2019-12-31 08:00:00+00:00,060372404021,58,8,1,1,13.793103,1.724138,1.724138,-1.610269,-1.421088,-0.900348
2999274,2019-12-31 08:00:00+00:00,060372404013,63,27,2,1,42.857143,3.174603,1.587302,1.251754,-1.159533,-0.935052
2999273,2019-12-31 08:00:00+00:00,060371380001,82,22,10,2,26.829268,12.195122,2.439024,-0.326558,0.467095,-0.719040
2999239,2019-12-31 08:00:00+00:00,060374026004,75,25,4,1,33.333333,5.333333,1.333333,0.313916,-0.770259,-0.999462


In [ ]:
# Normalize time
devices_mobility['date'] = devices_mobility['date'].dt.normalize()
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
0,2019-01-01 00:00:00+00:00,060371012201,213,100,10,1,46.948357,4.694836,0.469484,1.654628,-0.885396,-1.218549
5495,2019-01-01 00:00:00+00:00,060372656011,60,21,5,2,35.000000,8.333333,3.333333,0.478038,-0.229283,-0.492229
5494,2019-01-01 00:00:00+00:00,060372282101,104,48,7,1,46.153846,6.730769,0.961538,1.576391,-0.518266,-1.093756
5493,2019-01-01 00:00:00+00:00,060371211011,59,25,1,1,42.372881,1.694915,1.694915,1.204068,-1.426358,-0.907759
5492,2019-01-01 00:00:00+00:00,060590993102,70,24,2,1,34.285714,2.857143,1.428571,0.407700,-1.216779,-0.975308
...,...,...,...,...,...,...,...,...,...,...,...,...
2999275,2019-12-31 00:00:00+00:00,060372404021,58,8,1,1,13.793103,1.724138,1.724138,-1.610269,-1.421088,-0.900348
2999274,2019-12-31 00:00:00+00:00,060372404013,63,27,2,1,42.857143,3.174603,1.587302,1.251754,-1.159533,-0.935052
2999273,2019-12-31 00:00:00+00:00,060371380001,82,22,10,2,26.829268,12.195122,2.439024,-0.326558,0.467095,-0.719040
2999239,2019-12-31 00:00:00+00:00,060374026004,75,25,4,1,33.333333,5.333333,1.333333,0.313916,-0.770259,-0.999462


In [ ]:
# Check the number of rows for each CBG
print('Min: {}\nMax: {}'.format(min(set(devices_mobility.groupby('cbg').count()['date'])), max(set(devices_mobility.groupby('cbg').count()['date']))))

Min: 18
Max: 365


In [ ]:
for col in devices_mobility.columns:
    print('\n')
    print(col)
#     if devices_mobility[col].dtype
#     print(devices_mobility[col].dtype)
    print('Min: {}\nMax: {}'.format(devices_mobility[col].min(), devices_mobility[col].max()))



date
Min: 2019-01-01 00:00:00+00:00
Max: 2019-12-31 00:00:00+00:00


cbg
Min: 060371011101
Max: 060599901000


total
Min: 5
Max: 13425


completely_home
Min: 1
Max: 8305


part_time_work
Min: 1
Max: 601


full_time_work
Min: 1
Max: 279


percentage_completely_home
Min: 0.628930817610063
Max: 94.46320054017556


percentage_part_time_work
Min: 0.07530120481927711
Max: 57.14285714285714


percentage_full_time_work
Min: 0.06414368184733803
Max: 50.0


norm_completely_home
Min: -2.906584234865166
Max: 6.333557532462023


norm_part_time_work
Min: -1.7184152032370885
Max: 8.572307112492147


norm_full_time_work
Min: -1.321349979630751
Max: 11.343223008248279


In [ ]:
# Print all Null and NaN values
print('We have {} null values.'.format(len(devices_mobility[devices_mobility.isna().any(axis=1)])))

We have 0 null values.


In [ ]:
# Grouping based on day
grouped_dm = devices_mobility.groupby(devices_mobility['date']).mean()
grouped_dm

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
date,,,,,,,,,,
2019-01-01 00:00:00+00:00,91.966954,38.666383,3.353420,1.433362,40.957248,3.990702,2.048633,1.064666,-1.012369,-0.818050
2019-01-02 00:00:00+00:00,92.322522,35.975219,7.806730,4.359086,37.975806,9.085046,5.339572,0.771074,-0.093731,0.016587
2019-01-03 00:00:00+00:00,92.666302,34.168732,7.639213,3.843659,35.514339,8.621878,4.538849,0.528686,-0.177252,-0.186489
2019-01-04 00:00:00+00:00,92.929292,32.884097,7.273721,3.402746,33.547562,8.059543,3.976539,0.335012,-0.278655,-0.329101
2019-01-05 00:00:00+00:00,95.242498,39.216985,4.383307,1.759325,39.580239,4.760187,2.236390,0.929068,-0.873612,-0.770432
...,...,...,...,...,...,...,...,...,...,...
2019-12-27 00:00:00+00:00,79.706597,23.071194,6.675131,3.461912,28.817149,8.687396,4.812786,-0.130806,-0.165437,-0.117014
2019-12-28 00:00:00+00:00,81.264066,24.124316,4.197837,1.606878,29.264420,5.234282,2.363637,-0.086762,-0.788121,-0.738160
2019-12-29 00:00:00+00:00,80.848481,26.429040,4.723208,1.959538,32.753811,6.307020,3.129313,0.256849,-0.594679,-0.543971


In [ ]:
grouped_dm.reset_index()

,date,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
0,2019-01-01 00:00:00+00:00,91.966954,38.666383,3.353420,1.433362,40.957248,3.990702,2.048633,1.064666,-1.012369,-0.818050
1,2019-01-02 00:00:00+00:00,92.322522,35.975219,7.806730,4.359086,37.975806,9.085046,5.339572,0.771074,-0.093731,0.016587
2,2019-01-03 00:00:00+00:00,92.666302,34.168732,7.639213,3.843659,35.514339,8.621878,4.538849,0.528686,-0.177252,-0.186489
3,2019-01-04 00:00:00+00:00,92.929292,32.884097,7.273721,3.402746,33.547562,8.059543,3.976539,0.335012,-0.278655,-0.329101
4,2019-01-05 00:00:00+00:00,95.242498,39.216985,4.383307,1.759325,39.580239,4.760187,2.236390,0.929068,-0.873612,-0.770432
...,...,...,...,...,...,...,...,...,...,...,...
360,2019-12-27 00:00:00+00:00,79.706597,23.071194,6.675131,3.461912,28.817149,8.687396,4.812786,-0.130806,-0.165437,-0.117014
361,2019-12-28 00:00:00+00:00,81.264066,24.124316,4.197837,1.606878,29.264420,5.234282,2.363637,-0.086762,-0.788121,-0.738160
362,2019-12-29 00:00:00+00:00,80.848481,26.429040,4.723208,1.959538,32.753811,6.307020,3.129313,0.256849,-0.594679,-0.543971
363,2019-12-30 00:00:00+00:00,79.377187,22.368076,6.160714,3.188897,27.486791,7.830068,4.364988,-0.261810,-0.320035,-0.230583


In [ ]:
#Visualising mobility

title = '{} {}'.format(city,year)
alt.Chart(grouped_dm.reset_index()).mark_line().encode(
    x=alt.X('date', title='Date'),
    y=alt.Y(alt.repeat('row'), type='quantitative')
).properties(
    width=300,
    height=250
).repeat(
    row=['percentage_completely_home','percentage_part_time_work','percentage_full_time_work']
).interactive()

alt.RepeatChart(...)

![la-daily-2019](https://github.com/chouhandiksha/bigdataproject/raw/main/media/social-dist/la-daily-2019.png)

In [ ]:
#Adding month column for further usage
devices_mobility['month'] = devices_mobility['date'].dt.month
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work,month
0,2019-01-01 00:00:00+00:00,060371012201,213,100,10,1,46.948357,4.694836,0.469484,1.654628,-0.885396,-1.218549,1
5495,2019-01-01 00:00:00+00:00,060372656011,60,21,5,2,35.000000,8.333333,3.333333,0.478038,-0.229283,-0.492229,1
5494,2019-01-01 00:00:00+00:00,060372282101,104,48,7,1,46.153846,6.730769,0.961538,1.576391,-0.518266,-1.093756,1
5493,2019-01-01 00:00:00+00:00,060371211011,59,25,1,1,42.372881,1.694915,1.694915,1.204068,-1.426358,-0.907759,1
5492,2019-01-01 00:00:00+00:00,060590993102,70,24,2,1,34.285714,2.857143,1.428571,0.407700,-1.216779,-0.975308,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999275,2019-12-31 00:00:00+00:00,060372404021,58,8,1,1,13.793103,1.724138,1.724138,-1.610269,-1.421088,-0.900348,12
2999274,2019-12-31 00:00:00+00:00,060372404013,63,27,2,1,42.857143,3.174603,1.587302,1.251754,-1.159533,-0.935052,12
2999273,2019-12-31 00:00:00+00:00,060371380001,82,22,10,2,26.829268,12.195122,2.439024,-0.326558,0.467095,-0.719040,12
2999239,2019-12-31 00:00:00+00:00,060374026004,75,25,4,1,33.333333,5.333333,1.333333,0.313916,-0.770259,-0.999462,12


In [ ]:
#Grouping data on basis of month value
grouped_dm = devices_mobility.groupby([devices_mobility['month']]).mean()
grouped_dm

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
month,,,,,,,,,,
1,95.730155,35.680571,7.786807,3.790462,35.690534,8.441053,4.330764,0.546037,-0.209859,-0.239263
2,94.336387,34.351233,7.983371,3.904181,34.652279,8.885777,4.579683,0.443797,-0.129664,-0.176133
3,90.284791,30.704201,7.833073,4.109696,32.359511,8.979470,4.952156,0.218021,-0.112769,-0.081668
4,87.278149,27.623812,8.297202,4.584314,30.571254,9.589462,5.498292,0.041926,-0.002772,0.056841
5,86.723975,26.782047,8.711979,4.853114,29.702807,10.253137,5.935428,-0.043592,0.116906,0.167706
6,82.814826,25.349912,7.821000,4.066970,29.826315,9.545798,5.167929,-0.031430,-0.010645,-0.026944
7,81.156342,24.573682,7.623760,4.052766,29.678177,9.487063,5.259208,-0.046018,-0.021237,-0.003794
8,90.499792,27.453733,8.320794,4.502983,27.918253,9.982444,5.678911,-0.219323,0.068093,0.102649
9,102.358783,31.704745,9.590781,5.116633,28.121244,10.287088,5.794101,-0.199334,0.123028,0.131864


In [ ]:
d = []
for col in ['percentage_completely_home','percentage_part_time_work','percentage_full_time_work']:
        for m in range(1,13):
            d.append({'month': m, 'column': col, 'value': grouped_dm[col][m]})
d = alt.Data(values=d)

In [ ]:
#Visualising percentage_devices mobility

title = '{} {}'.format(city,year)
alt.Chart(d, title=title).mark_line().encode(
    x=alt.X('month:O', title='Month'),
    y=alt.Y('value:Q', type='quantitative', title='Percentage'),
    color='column:N'
).properties(
    width=300,
    height=250
).interactive()

alt.Chart(...)

![la-monthly-2019](https://github.com/chouhandiksha/bigdataproject/raw/main/media/social-dist/la-monthly-2019.png)

**Get Stats**

In [ ]:
df_summary = devices_mobility.describe()
df_summary

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work,month
count,3.004769e+06,3.004769e+06,3.004769e+06,3.004769e+06,3.004769e+06,3.004769e+06,3.004769e+06,3.004769e+06,3.004769e+06,3.004769e+06,3.004769e+06
mean,8.784796e+01,2.776353e+01,8.179268e+00,4.286186e+00,3.014549e+01,9.604832e+00,5.274169e+00,2.506247e-12,2.150548e-12,-6.891095e-14,6.525355e+00
std,1.055159e+02,4.993209e+01,8.106741e+00,4.523167e+00,1.015507e+01,5.545535e+00,3.942956e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.447803e+00
min,5.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.289308e-01,7.530120e-02,6.414368e-02,-2.906584e+00,-1.718415e+00,-1.321350e+00,1.000000e+00
25%,5.000000e+01,1.300000e+01,3.000000e+00,1.000000e+00,2.307692e+01,5.263158e+00,2.150538e+00,-6.960632e-01,-7.829136e-01,-7.922054e-01,4.000000e+00
50%,7.100000e+01,2.100000e+01,6.000000e+00,3.000000e+00,2.985075e+01,8.917197e+00,4.166667e+00,-2.902449e-02,-1.239979e-01,-2.808812e-01,7.000000e+00
75%,1.020000e+02,3.200000e+01,1.100000e+01,6.000000e+00,3.684211e+01,1.304348e+01,7.471264e+00,6.594356e-01,6.200748e-01,5.572204e-01,1.000000e+01
max,1.342500e+04,8.305000e+03,6.010000e+02,2.790000e+02,9.446320e+01,5.714286e+01,5.000000e+01,6.333558e+00,8.572307e+00,1.134322e+01,1.200000e+01


In [ ]:
path = Path('drive/MyDrive/big-data-project/analysis/')
path.mkdir(parents=True, exist_ok=True)
filename = path/'{}_social_summary_{}.csv'.format(city_directory, year)
df_summary.to_csv(filename)